# rospy Snippets

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/Foundations-of-Robotics/foundations-of-robotics-labs/blob/master/1-sense-think-act/02-rospy-snippets.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Foundations-of-Robotics/foundations-of-robotics-labs/master?filepath=1-sense-think-act/02-rospy-snippets.ipynb)

The Sphero RVR is a great robot for learning some robotics, but how do we work with ROS? This tutorial will show you how to use snippets of ROS code using rospy in python to interact with the RVR.

## Receiving Messages

A core part of ROS programming is the use of ***`callback`*** functions. In ROS, A callback function is executed when a message is received on a topic. This is common place in many programs that perform communications. Messaging applications such as Messenger, Slack, or Discord also use a callback pattern pass to messages around the world.

The following code snippet shows how to create a callback function, subscribe to a topic, and execute the callback function when a message is received.

In [ ]:
"""receiving message data"""


# import python libraries
import rospy # ros library for python


# create a class to subscribe to sensor messages and show the data
class SubscribeAndShowNode():
    def __init__(self):
        # initialize the node with rospy
        rospy.init_node('subscribe_and_show_node')
        # subscribe to the topic /sensor_data
        rospy.Subscriber('/sensor_data', String, self.callback)
        # spin() simply keeps python from exiting until this node is stopped
        rospy.spin()

    # callback function to receive the data
    def callback(self, data):
        # print the data received
        rospy.loginfo(rospy.get_caller_id() + 'I heard %s', data.data)


# main function
def main():
    try:
        SubscribeAndShowNode()
    except rospy.ROSInterruptException:
        pass

#### Test out subscribing to a topic

Run the following code snippet to see how to subscribe to a topic and print out the message. Inspect the ROS Network Graph to see the node that was created and the topic it is subscribing to.

In [ ]:
"""run the main function to start the node and test it"""


# import a user interface library
import ipywidgets as widgets


# create a button to start the node
button = widgets.Button(description="Start node")
# create a button to stop the node
button_stop = widgets.Button(description="Stop node")

# create a function to start the node
def start_node(button):
    # start the node
    main()
    # disable the start button
    button.disabled = True
    # enable the stop button
    button_stop.disabled = False

# create a function to stop the node
def stop_node(button):
    # stop the node
    rospy.signal_shutdown('subscribe_and_show_node')
    # disable the stop button
    button.disabled = True
    # enable the start button
    button_stop.disabled = False

# add the start_node function to the button
button.on_click(start_node)
# add the stop_node function to the button
button_stop.on_click(stop_node)

# display the buttons
display(button, button_stop)

## Sending Commands

Sending commands in a ROS application is very similar to receiving messages but instead of reveiving messages, you are sending them. Another component in the system will listen for and receive these messages. The following code snippet shows how to create a ***`publisher`***, publish a message, and shutdown the publisher.

In [ ]:
"""sending commands to the robot to move it"""


# import python libraries
import rospy # ros library for python
import


# create a class to send commands to the robot


## ROS-Style Loops

In robotics it is common to have a loop of activities that runs continuously at regular intervals. This is often referred to as a loop in programming. The best practice for creating loops in a ROS application is to use a ***`Timer`***. The following code snippet shows how to create a ROS-Style loop using a ROS Timer.

In [ ]:
"""timer based loop"""


import rospy # ros library for python


# create a class to send commands to the robot
class TimerBasedLoopNode():
    def __init__(self):
        # initialize the node with rospy
        rospy.init_node('timer_based_loop_node')
        # create a timer to call the callback function every 1 second
        self.timer = rospy.Timer(rospy.Duration(1.0), self.callback)
        # spin() simply keeps python from exiting until this node is stopped
        rospy.spin()

    # callback function to send commands to the robot
    def callback(self, event):
        # create a publisher to send commands to the robot
        pub = rospy.Publisher('/robot_commands', String, queue_size=10)
        # create a string message
        msg = String()
        # add the command to the message
        msg.data = 'move forward'
        # publish the message
        pub.publish(msg)

## Putting it all together

Now that we have seen how to receive messages, send commands, and create ROS-style loops, we can put it all together to create a simple ROS program. The following code snippet shows how to create a ROS node that receives messages from the RVR's IMU and sends commands to the RVR's motors.